In [8]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snsf
from datetime import datetime
from functools import reduce
# from bokeh.io import output, output_file, output_notebook, reset_output
from bokeh.plotting import figure, show, output_file, save, output_notebook
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import time
import random
import re
from datetime import date
import os
import glob
import regex as re
import numpy as np
from bokeh.models import HoverTool
from bokeh.models import Span


In [24]:
region = input("Enter Region")
if region == "south":
    cities = ["bangalore", "chennai", "hyderabad", "thiruvananthapuram"]
elif region == "north":
    cities = ["jaipur", "jalandhar", "ludhiana", "newdelhi", "bikaner"]
elif region == "east":
    cities = ["bhubaneswar", "kolkata", "guwahati"]
elif region == "west":
    cities = ["mumbai", "kota", "indore" ]
elif region == "northeast":
    cities = ["jaipur", "jalandhar", "ludhiana", "newdelhi", "bhubaneswar", "kolkata", "guwahati"]
elif region == "delhi":
    cities = ["newdelhi"]
elif region == "all":
    cities = ["jaipur", "jalandhar", "ludhiana", "newdelhi", "bhubaneswar", "kolkata", "guwahati", "mumbai", "kota", "indore","bangalore", "chennai", "hyderabad", "thiruvananthapuram"]
    
colnames = ["date"] + cities + ['rf' + city for city in cities] + ["averagemin"] + ['realfeelavg']
years = [2017, 2018, 2019, 2020]
years2 = ['2017avg', '2018avg', '2019avg', '2020avg']
mydateparser = lambda x: pd.datetime.strptime(x, "%y %m %d")
for year in years:
#     dfyear = "df"+str(year)
    dfyear = pd.read_csv('ludhiana'+"-"+str(year)+".csv", parse_dates=['date'])
    dfyear = dfyear[['date']]
    for city in cities:
#         dfcityyear = "df"+city+str(year)
        dfcityyear = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/"+city+"-"+str(year)+ ".csv", parse_dates=['date'])
        dfcityyear = dfcityyear[['tmin','realfeelmin']]
        dfyear = pd.concat([dfyear, dfcityyear], axis=1, join="outer")
    dfyear['averagemin'] = dfyear.iloc[:,[1,3,5]].mean(axis=1)
    dfyear['realfeelavg'] = dfyear.iloc[:,[2,4,6]].mean(axis=1)
    dfyear.columns = colnames
    globals()["df"+str(year)] = dfyear
    
dfpast = pd.concat([df2017['date'],df2017['averagemin'],df2018['averagemin'],df2019['averagemin'],df2020['averagemin'],df2017['realfeelavg'],df2018['realfeelavg'],df2019['realfeelavg'],df2020['realfeelavg']], axis=1)
dfpast.columns=['date'] + years+years2
for cols in dfpast.columns.tolist()[1:]:
    dfpast = dfpast[dfpast[cols] > 0]
# cities = ["jaipur", "jalandhar", "ludhiana", "new-delhi"]
colnames = ["date"] + cities + ["2021"]+['realfeel']
years = [2021]
for year in years:
#     dfyear = "df"+str(year)
    dfyear = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/"+city+"-"+str(year)+".csv", parse_dates=['date'])
    dfyear = dfyear[['date']]
    for city in cities:
#         dfcityyear = "df"+city+str(year)
        dfcityyear = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/"+city+"-"+str(year)+".csv", parse_dates=['date'])
        dfcityyear = dfcityyear[['tmin', 'realfeelmin']]
        dfyear = pd.concat([dfyear, dfcityyear], axis=1, join="outer")
    dfyear['2021'] = dfyear[['tmin']].mean(axis=1)
    dfyear['realfeel'] = dfyear[['realfeelmin']].mean(axis=1)    
#     dfyear.columns = colnames
    globals()["df"+str(year)]=dfyear
df2021 = df2021[['date', '2021', 'realfeel']]
dfpast['MonthDay'] = dfpast['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
df2021['MonthDay'] = df2021['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
maindf=pd.merge(dfpast, df2021, on="MonthDay", how="inner")
maindf.drop(['date_x'], axis=1, inplace=True)
maindf.rename(columns={'date_y':'date'}, inplace=True)
maindf.drop(['MonthDay'], axis=1, inplace=True)
column_to_move = maindf.pop("date")
maindf.insert(0, "date", column_to_move)
df2 = maindf.groupby(pd.Grouper(freq='W', key='date'))[2017,2018,2019,2020,'2021','realfeel','2017avg', '2018avg', '2019avg', '2020avg'].mean()
df2.reset_index(inplace=True)
for a,b,c in zip(df2['2021'], df2['realfeel'], df2['date']): 
    if b<a:
        break
    else:
        continue
df2['thresh'] = a
df2['dateeq'] = c
df3=pd.read_csv("C://Users/Akash/New folder (4)/searches/Searches.csv")
df2['searchterm']=df3['scaledown']
df2.dropna()
dates = list(df2['date'])
dates = [x.strftime("%Y-%m-%d") for x in dates]
p = figure(title=region, x_axis_type= "datetime", plot_width =800,plot_height=600)
# p.line(df1['date'], df1[''].tolist(), legend_label="2017", line_color="black", line_width=2)
# p.line(df2['date'], df2[2018].tolist(), legend_label="2018", line_color="brown", line_width=2)
# p.line(df2['date'], df2[2019], line_color="blue", legend_label = "2019")
# p.line(df2['date'], df2[2020].tolist(), legend_label="2020", line_color="red", line_width=2)
p.line(df2['date'], df2['2021'], legend_label="2021", line_color="green", line_width=2)
p.line(df2['date'], df2['realfeel'], legend_label = "Realfeel", line_color="black")
# p.line(df2['date'], df2['2020avg'], legend_label = "realfeel2020", line_color="pink")
p.line(df2['date'], df2['searchterm'], legend_label="searchterm", line_color="magenta", line_width=2)
p.line(df2['date'], df2['thresh'], legend_label = "Threshold", line_color="brown")
date_eq = Span(location=c,
                            dimension='height', line_color='red',
                            line_dash='dashed', line_width=3)
p.add_layout(date_eq)
p.yaxis.axis_label = 'Temperature'
p.xaxis.axis_label = 'Date'
p.add_tools(HoverTool(tooltips=[('date', '$x{%F}'), ('mintemp', '$y')],formatters = {'$x':'datetime'}))
# HoverTool(tooltips=[('date', '@dates{%F}')],formatters = {'@dates':'datetime'})
show(p)
output_file('C://Users/Akash/New folder (4)/Untitled Folder/graphs/' + region + '.html')
save(p)
print(df2[['searchterm', 'realfeel']].corr(method='pearson'))
print(df2[[2018, '2021']].corr(method='pearson'))
print(df2[[2019, '2021']].corr(method='pearson'))
print(df2[[2020, '2021']].corr(method='pearson'))
print("Date of winter onset =", c)

Enter Regionnorth


C:\Users\Akash\AppData\Local\Temp/ipykernel_21348/956982939.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2021['MonthDay'] = df2021['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
C:\Users\Akash\AppData\Local\Temp/ipykernel_21348/956982939.py:64: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = maindf.groupby(pd.Grouper(freq='W', key='date'))[2017,2018,2019,2020,'2021','realfeel','2017avg', '2018avg', '2019avg', '2020avg'].mean()


            searchterm  realfeel
searchterm    1.000000 -0.983858
realfeel     -0.983858  1.000000
       2018   2021
2018  1.000  0.993
2021  0.993  1.000
          2019      2021
2019  1.000000  0.970144
2021  0.970144  1.000000
          2020      2021
2020  1.000000  0.972697
2021  0.972697  1.000000
Date of winter onset = 2021-10-17 00:00:00


In [21]:
dfyear.iloc[:,[1,3,5,7,9]]

,tmin,tmin,tmin,tmin,tmin
0,27.22,25.56,25.56,27.78,27.78
1,26.11,26.11,26.11,27.22,29.44
2,22.78,28.33,28.33,29.44,31.67
3,27.78,27.78,27.78,30.56,32.22
4,26.11,29.44,29.44,29.44,31.67
...,...,...,...,...,...
148,NaN,6.67,6.67,6.11,8.89
149,NaN,5.56,5.56,5.56,7.22
150,NaN,NaN,NaN,5.56,6.11
151,NaN,NaN,NaN,7.22,7.22


In [274]:
output_file('C://Users/Akash/New folder (4)/Untitled Folder/graphs/south.html')
save(p)

NameError: name 'save' is not defined

In [271]:
%pwd

'C:\\Users\\Akash\\New folder (4)'

In [13]:
cities = ["jaipur", "jalandhar", "ludhiana", "new-delhi"]
colnames = ["date"] + cities + ["2021"]+['realfeel']
years = [2021]
for year in years:
#     dfyear = "df"+str(year)
    dfyear = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/"+city+"-"+str(year)+".csv", parse_dates=['date'])
    dfyear = dfyear[['date']]
    for city in cities:
#         dfcityyear = "df"+city+str(year)
        dfcityyear = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/"+city+"-"+str(year)+".csv", parse_dates=['date'])
        dfcityyear = dfcityyear[['tmin', 'realfeelmin']]
        dfyear = pd.concat([dfyear, dfcityyear], axis=1, join="outer")
    dfyear['2021'] = dfyear[['tmin']].mean(axis=1)
    dfyear['realfeel'] = dfyear[['realfeelmin']].mean(axis=1)    
#     dfyear.columns = colnames
    globals()["df"+str(year)]=dfyear
df2021 = df2021[['date', '2021', 'realfeel']]
dfpast['MonthDay'] = dfpast['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
df2021['MonthDay'] = df2021['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
maindf=pd.merge(dfpast, df2021, on="MonthDay", how="inner")
maindf.drop(['date_x'], axis=1, inplace=True)
maindf.rename(columns={'date_y':'date'}, inplace=True)
maindf.drop(['MonthDay'], axis=1, inplace=True)
column_to_move = maindf.pop("date")
maindf.insert(0, "date", column_to_move)
df2 = maindf.groupby(pd.Grouper(freq='W', key='date'))[2017,2018,2019,2020,'2021', 'realfeel'].mean()
df2.reset_index(inplace=True)
df2.dropna()
p = figure(title="temp 3 years", x_axis_type= "datetime", plot_width =1200,plot_height=600)
# p.line(df1['date'], df1[''].tolist(), legend_label="2017", line_color="black", line_width=2)
p.line(df2['date'], df2[2018].tolist(), legend_label="2018", line_color="brown", line_width=2)
p.line(df2['date'], df2[2019], line_color="blue", legend_label = "2019")
p.line(df2['date'], df2[2020].tolist(), legend_label="2020", line_color="red", line_width=2)
p.line(df2['date'], df2['2021'], legend_label="2021", line_color="green", line_width=2)
p.line(df2['date'], df2['searchterm'], legend_label="searchterm", line_color="magenta", line_width=2)
p.line(df2['date'], df2['realfeel'], legend_label = "realfeel", line_color="black")
show(p)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Akash/New folder (4)/Untitled Folder/north/new-delhi-2021.csv'

In [65]:
df = pd.read_csv("C://Users/Akash/New folder (4)/Untitled Folder/north/bikaner-2021.csv")

In [106]:
maindf=pd.merge(dfpast, df2021, on="date", how="inner")

In [122]:
df2021 = df2021[['date', '2021']]
dfpast['MonthDay'] = dfpast['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
df2021['MonthDay'] = df2021['date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
maindf=pd.merge(dfpast, df2021, on="MonthDay", how="inner")

X:\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [167]:
maindf[2017]
# column_to_move = maindf.pop("date")
# maindf.insert(0, "date", column_to_move)

0     26.332
1     26.110
2     25.666
3     24.002
4     24.224
       ...  
86    11.222
87    12.112
88    12.446
89    11.556
90    10.000
Name: 2017, Length: 91, dtype: float64

In [36]:
df2

,date,2017,2018,2019,2020,2021,realfeel,2017avg,2018avg,2019avg,2020avg,thresh
0,2021-09-19,24.537500,23.840833,25.185000,27.221667,24.714286,28.714286,21.346693,20.924994,21.355822,24.164596,20
1,2021-09-26,24.099524,22.990952,24.577143,25.635714,24.918367,28.755102,20.873853,19.961822,20.633266,22.060044,20
2,2021-10-03,23.756667,22.936190,22.672381,22.540000,23.897959,26.918367,20.260773,19.847574,18.703338,18.718649,20
3,2021-10-10,22.354286,21.746190,20.899048,21.800000,23.306122,25.816327,18.714932,18.805692,17.196616,18.332734,20
4,2021-10-17,21.215714,19.339524,21.323810,20.370476,21.244898,21.857143,17.730373,16.660388,17.513058,17.372820,20
5,2021-10-24,18.518095,17.303333,18.359524,17.752857,19.918367,21.020408,15.457683,15.041953,15.253520,15.153778,20
6,2021-10-31,17.302857,17.461429,18.465238,14.708095,18.612245,18.897959,14.478466,15.185561,15.345563,12.675006,20
7,2021-11-07,16.827143,15.767619,17.514286,13.940952,16.653061,16.040816,14.057032,13.588456,14.599265,12.058897,20
8,2021-11-14,13.756667,14.762381,15.397619,13.967619,15.081633,14.693878,10.919007,12.810707,12.778044,12.073934,20
9,2021-11-21,11.719048,14.020952,14.576667,11.720000,14.612245,13.857143,8.870236,12.155741,12.059480,9.848757,20


In [256]:
dates = list(df2['date'])

In [259]:
dates

['2021-09-19',
 '2021-09-26',
 '2021-10-03',
 '2021-10-10',
 '2021-10-17',
 '2021-10-24',
 '2021-10-31',
 '2021-11-07',
 '2021-11-14',
 '2021-11-21',
 '2021-11-28',
 '2021-12-05',
 '2021-12-12',
 '2021-12-19']

In [10]:
df2
df3=pd.read_csv("C://Users/Akash/New folder (4)/searches/Searches.csv")

In [11]:
df2['searchterm']=df3['Search']

In [12]:
df2

,date,2017,2018,2019,2020,2021,realfeel,2017avg,2018avg,2019avg,2020avg,thresh,searchterm
0,2021-09-19,24.704000,24.146667,25.296000,27.406667,24.480000,28.760000,21.548623,21.286428,21.542728,24.420317,20,28.000000
1,2021-09-26,24.099524,22.990952,24.577143,25.635714,23.885714,26.885714,20.873853,19.961822,20.633266,22.060044,20,32.405736
2,2021-10-03,23.756667,22.936190,22.672381,22.540000,22.800000,25.028571,20.260773,19.847574,18.703338,18.718649,20,34.783133
3,2021-10-10,22.354286,21.746190,20.899048,21.800000,21.714286,23.657143,18.714932,18.805692,17.196616,18.332734,20,33.825793
4,2021-10-17,21.215714,19.339524,21.323810,20.370476,18.685714,18.371429,17.730373,16.660388,17.513058,17.372820,20,49.186903
5,2021-10-24,18.518095,17.303333,18.359524,17.752857,17.085714,17.057143,15.457683,15.041953,15.253520,15.153778,20,51.212365
6,2021-10-31,17.302857,17.461429,18.465238,14.708095,15.714286,15.428571,14.478466,15.185561,15.345563,12.675006,20,55.285936
7,2021-11-07,16.827143,15.767619,17.514286,13.940952,13.571429,13.171429,14.057032,13.588456,14.599265,12.058897,20,55.670642
8,2021-11-14,13.756667,14.762381,15.397619,13.967619,12.285714,12.257143,10.919007,12.810707,12.778044,12.073934,20,61.018979
9,2021-11-21,11.719048,14.020952,14.576667,11.720000,12.228571,11.657143,8.870236,12.155741,12.059480,9.848757,20,61.021731


In [41]:
for a,b in zip(df2['2021'], df2['realfeel']): 
    if b<a:
        break
    else:
        continue

In [43]:
b

18.04081632653061

In [49]:
df2[['searchterm', 'realfeel']].corr(method='pearson')

,searchterm,realfeel
searchterm,1.000000,-0.983858
realfeel,-0.983858,1.000000
